In [7]:
import os
import numpy as np

data = 'archive/Garbage classification/Garbage classification'

print(os.listdir(data))
from PIL import Image

png_filepath = data + '/paper/paper312.jpg'
png_pil_img = Image.open(png_filepath)
png_np_img = np.asarray(png_pil_img)

['paper', '.DS_Store', 'metal', 'cardboard', 'trash', 'glass', 'plastic']


In [16]:
png_np_img.T.shape

(3, 512, 384)

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 64 # Make this number of images or do actual BGD
target_size = (128, 128) # Make this smaller

train = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.1,zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

test =ImageDataGenerator(rescale=1./255,validation_split=0.1)

train_generator = train.flow_from_directory(
    data,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset = 'training'
)

test_generator = test.flow_from_directory(
    data,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset = 'validation'
)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


In [34]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models, layers
import tensorflow as tf

def create_model():
    labels = list(train_generator.class_indices.keys())
    print(labels)
    num_classes = len(labels)

    # Build Model
    model = models.Sequential()


    model.add(layers.Conv2D(8,(3,3), padding='same', input_shape=(128, 128,3),activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2))) 

    model.add(layers.Conv2D(8,(3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2))) 

    model.add(layers.Conv2D(16,(3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Conv2D(16,(3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2))) 

    model.add(layers.Conv2D(32,(3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Conv2D(32,(3,3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2))) 


    model.add(layers.Flatten())
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(num_classes,activation='softmax'))

    # adding a dropout layer, batch normalization, early stopping
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    return model, checkpoint

In [36]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import ModelCheckpoint

# Create your model and checkpoint
model, checkpoint = create_model()

# Train the model using the training generator
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Epoch 1/10
2/2 [==============================] - 19s 16s/step - loss: 1.7725 - accuracy: 0.2144 - val_loss: 1.7450 - val_accuracy: 0.2351
Epoch 2/10
2/2 [==============================] - 17s 1s/step - loss: 1.7455 - accuracy: 0.2351 - val_loss: 1.7275 - val_accuracy: 0.2351
Epoch 3/10
2/2 [==============================] - 18s 16s/step - loss: 1.7263 - accuracy: 0.2351 - val_loss: 1.7207 - val_accuracy: 0.1992
Epoch 4/10
2/2 [==============================] - 18s 16s/step - loss: 1.7260 - accuracy: 0.1982 - val_loss: 1.7202 - val_accuracy: 0.1992
Epoch 5/10
2/2 [==============================] - 18s 16s/step - loss: 1.7223 - accuracy: 0.2280 - val_loss: 1.7148 - val_accuracy: 0.2351
Epoch 6/10
2/2 [==============================] - 18s 16s/step - loss: 1.7194 - accuracy: 0.2351 - val_loss: 1.7151 - val_accuracy: 0.2351
Epoch 7/10
2/2 [==============================] - 17s 15s/step - loss: 1.7204 - accuracy: 0.2351 - val_los

In [ ]:
'''
Optimization
- add dropout, batch = 64, l2 weight normalization
''''''